<a href="https://colab.research.google.com/github/np2802/Indian-Legal-Semantic-Searcher/blob/main/CaseSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch

# Check if GPU is available and set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cpu


In [ ]:
import spacy

# Increase max length limit in spacy
nlp = spacy.blank('en')
nlp.max_length = 5000000

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
import time
import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration
import torch


# Path to the CSV file
csv_file_path = '/content/drive/MyDrive/FYP/Topic_Modelling_New/topic_assignments_with_content.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Load BART model and tokenizer
model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name).to(device)

# Function to summarize content
def summarize_content(content):
    # Tokenize input
    inputs = tokenizer.encode("summarize: " + content, return_tensors="pt", max_length=1024, truncation=True).to(device)

    # Generate summary
    summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

# Apply summarization to each row in the 'Content' column and create a new 'Summary' column
def process_dataframe(df):
    summaries = []
    for index, row in df.iterrows():
        content = row['Content']
        file_name = row['File Name']
        print(f"Summarizing document: {file_name}")

        # Record start time
        start_time = time.time()

        # Summarize the content
        summary = summarize_content(content)

        # Record end time
        end_time = time.time()

        # Calculate and print time taken
        time_taken = end_time - start_time
        print(f"Time taken to summarize document {file_name}: {time_taken:.2f} seconds")

        summaries.append(summary)

    df['Summary'] = summaries
    return df

# Process the DataFrame
df = process_dataframe(df)

# Save the updated DataFrame back to CSV
output_csv_file_path = '/content/drive/MyDrive/FYP/Dataset/TopicAssignment_with_summaries.csv'
df.to_csv(output_csv_file_path, index=False)

print(f"Summaries saved to {output_csv_file_path}")
